In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

AI_CELL = 'Cpaaa'
TARGET_CELL = 'ABarpaapp'
RESOLUTION = 0.254
def load_data(embryo):
    """
    input:
        embryo: int, choose embryo from 1-6
    output:
        data_dicts:dict,dict[nuclei stage][name key][pos/radius],containing postion, radius, cell name
        ai_first_appear: int, 'Cpaaa' first appear nuclei stage
        ai_last_appear: int, 'ABarpaapp' first appear nuclei stage
        target_first_appear: int, 'Cpaaa' die nuclei stage
        target_last_appear: int, 'ABarpaapp' die nuclei stage
    """
    pos_a = []
    radius_a =[]
    cell_name_a = []
    data_dicts = []
    ai_list = []
    target_list = []
    nuclei_num = len(os.listdir('./embryo_data/%02d/nuclei' % embryo))
    for i in range (nuclei_num - 1):
        path = './embryo_data/%02d/nuclei/t%03d-nuclei' % (embryo,i + 1)
        pos = []
        radius = []
        cell_name = []
        with open(path) as file:
            for line in file:
                line = line[:len(line)-1]
                vec = line.split(', ')
                if vec[9] == '':
                    continue
                else:
                    id = int(vec[0])
                    pos.append(np.array(((float(vec[5])*RESOLUTION), (float(vec[6])*RESOLUTION), (float(vec[7])))))
                    radius.append(float(vec[8]) / 2)
                    cell_name.append(vec[9])
        pos_a.append(pos)
        radius_a.append(radius)
        cell_name_a.append(cell_name)
        pos_radius = list(zip(pos, radius))
        data_dict = dict(zip(cell_name, pos_radius))
        data_dicts.append(data_dict)
    
    for i in range(len(cell_name_a)):
        if AI_CELL in cell_name_a[i]:
            ai_list.append(i)
        if TARGET_CELL in cell_name_a[i]:
            target_list.append(i)
    if ai_list != []:
        ai_first_appear = ai_list[0]
        target_first_appear = target_list[0]
        ai_last_appear = ai_list[-1]
        target_last_appear = target_list[-1]
    else:
        ai_first_appear = 0
        target_first_appear = 0
#     np.save('original ai location', pos_a[168:196])
    return data_dicts,ai_first_appear,ai_last_appear,target_first_appear,target_last_appear

In [ ]:
#######Single Embryo distance figure######
def Embryo_dis(embryo):
    """
    input:
        embryo: int, choose embryo from 1-6
    output:
        Distance_to_target
    """
    if embryo == 0:
        time_resolution = 1
        shift_point = 25
    else: 
        time_resolution = 1.25
        shift_point = 20
    
    data_dicts,ai_first_appear,ai_last_appear,target_first_appear,target_last_appear = load_data(embryo)
    ####Starting and ending stage, starting point and the end points (x,y,z) of these two cells
    print('AI cell \nappear stage: (%s,%s)' % (ai_first_appear,ai_last_appear))
    print('starting location {} \nending location   {}'.format(data_dicts[ai_first_appear][AI_CELL][0],data_dicts[ai_last_appear][AI_CELL][0]))
    print('\nTARGET cell \nappear stage: (%s,%s)'% (target_first_appear,target_last_appear))
    print('starting location {} \nending location   {}'.format(data_dicts[target_first_appear][TARGET_CELL][0],data_dicts[target_last_appear][TARGET_CELL][0]))
    ####Maximum distance using starting stage of AI cell
    pos = []
    
    for key in data_dicts[target_first_appear].keys():
        pos.append(data_dicts[target_first_appear][key][0])
    distances = []
    for i in range(len(pos)):
        for j in range(len(pos)):
            distances.append(np.linalg.norm(pos[i]-pos[j])) 
    print("\nMaximum distance: %d"%max(distances))
    
    ###Distance to target
    distance_to_target = []
    begin = max(ai_first_appear,target_first_appear)
#     begin = ai_first_appear + 15
    difference = target_first_appear - ai_first_appear
    end = min(ai_last_appear,target_last_appear)
    print(begin,end,difference)
    for i in range(end-begin):
        distance_to_target.append(np.linalg.norm(data_dicts[begin + i][AI_CELL][0]- \
                                        data_dicts[begin + i][TARGET_CELL][0]))        
    ##shifting range
    distance_to_target_mid = (max(distance_to_target) + min(distance_to_target)) / 2
    distance_to_target_mid_index = np.argmin(np.abs(np.array(distance_to_target)-distance_to_target_mid))
    print("average_y_mid: {}\ny_mid_index: {}\nactual_y_mid[y_mid_index]: {}\n".format(distance_to_target_mid, distance_to_target_mid_index,distance_to_target[distance_to_target_mid_index]))
    shift_num = distance_to_target_mid_index - shift_point
    print("shift step: %f"% shift_num)
    
    ###Draw distance figure
    fig = plt.figure(1)
    plt.title("Distance to target cell over time")
    plt.xlabel("Time(s)")
    plt.ylabel("Distance to target cell(um)")
    plt.plot([time_resolution * k for k in range(len(distance_to_target))],distance_to_target)
    plt.show()
    return distance_to_target, shift_num, distance_to_target_mid_index, difference


In [ ]:
###Calculate each cell's shitfting number and distance to target
time_resolution_2D = 1.25
distances = []
shift_nums = []
distance_to_target_mid_indexs = []
differences = []
for i in range(1,7):
    distance, shift_num, distance_to_target_mid_index, difference = Embryo_dis(i)
    distances.append(distance)
    shift_nums.append(shift_num)
    distance_to_target_mid_indexs.append(distance_to_target_mid_index)
    differences.append(difference)

In [ ]:
#Plot raw distance to target plot
fig = plt.figure(figsize=(15,15))
for i in range(len(distances)):
    plt.plot([time_resolution_2D * k for k in range(len(distances[i]))],distances[i], linewidth = 5)
for i in range(len(distances)):
    plt.scatter(distance_to_target_mid_indexs[i] * time_resolution_2D,distances[i][distance_to_target_mid_indexs[i]], c = 'r',s = 600)

plt.scatter(distance_to_target_mid_indexs[0] * time_resolution_2D,distances[0][distance_to_target_mid_indexs[0]], c = 'r',s = 400,label = 'Midpoint')
# plt.title("Distance to target cell over time")
plt.xlabel("Time (minutes)", fontsize = 45)
plt.ylabel("Distance to Target (um)",fontsize = 45)
plt.xticks(ticks = np.arange(0,45,2),labels = np.arange(-15,30,2),fontsize=24)
plt.yticks(ticks = np.arange(0,24),labels = np.arange(0/0.254,24/0.254,1/0.254).astype(int),fontsize=24)
plt.yticks(fontsize=24)
plt.xlim(xmin =15, xmax = 38)
plt.ylim(ymin = 8, ymax = 24)
plt.legend(fontsize = 45)
plt.show()

In [ ]:
import statistics 
distances_shifted = []
for i in range(len(distances)):
    distances_shifted.append([0 for i in range(abs(shift_nums[i]))] + distances[i])
distances_avg = []
distances_stdev = []
distances_upper = []
distances_lower = []
point_num = min([len(k) for k in distances_shifted])
set_num = len(distances_shifted)
for i in range(point_num):
    temp = []
    for j in range(set_num):
        temp.append(distances_shifted[j][i])
    distances_avg.append(statistics.mean(temp))
    distances_stdev.append(statistics.stdev(temp))
    distances_upper.append(distances_avg[i] + distances_stdev[i])
    distances_lower.append(distances_avg[i] - distances_stdev[i])

In [ ]:
#Plot normalized distance to target plot
fig = plt.figure(figsize=(15,15))
plt.fill_between([time_resolution_2D * k for k in range(len(distances_lower))], \
                 distances_upper, distances_lower, facecolor='silver', interpolate=True)


plt.plot([time_resolution_2D * k for k in range(len(distances_avg))],distances_avg, label = 'Average', linewidth=7)
plt.xticks(ticks = np.arange(0,45,2),labels = np.arange(-15,30,2),fontsize = 24)
plt.yticks(ticks = np.arange(0,24),labels = np.arange(0/0.234,24/0.234,1/0.233).astype(int),fontsize = 24)
plt.xlim(xmin =15, xmax = 38)
plt.ylim(ymin = 8, ymax = 24)
plt.xlabel('Time (minutes)',fontsize = 45)
plt.ylabel('Distance to Target (um)',fontsize = 45)
plt.legend(loc = 'upper right',fontsize = 35)